In [1]:
from ScrapeFB import *
import pandas as pd
import bs4, time, re, json
from datetime import datetime, timedelta

In [2]:
window = WindowNoLogin(url='https://www.facebook.com/FreeYOUTHth/')

In [3]:
for i in range(50):
    window.scrape(filename='freeyouthFB.json', scroll=2)

100%|██████████| 2/2 [00:11<00:00,  5.76s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#pagelet_timeline_main_column"}
  (Session info: chrome=88.0.4324.96)


In [16]:
len(window.scraped_posts)

823

In [2]:
window = WindowLogin(
    url='https://www.facebook.com/FreeYOUTHth/',
    browser='Firefox',
)

In [3]:
window.scrape(scroll=50)

100%|██████████| 50/50 [01:45<00:00,  2.12s/it]


# show data

In [4]:
df = pd.read_json('FB.json')
df

,date,article,url,comment,share,hashtags
0,2021-08-01 21:14:00,ไล่ยิงประชาชนจนตอนนี้ยังไม่หยุด! กระสุนทุกลูก ...,https://www.facebook.com/FreeYOUTHth/posts/560...,3.1 พัน,1.4 หมื่น,[#ม็อบ7สิงหา]
1,2021-08-01 22:11:00,พวกมึงจะเอาให้ตายเลยใช่มั้ย? \n\n\nขอบคุณภาพจา...,https://www.facebook.com/FreeYOUTHth/posts/560...,3 พัน,2.1 หมื่น,[#ไอ้ฆาตกร]
2,2021-08-01 20:00:00,ยิงได้ยิงดี 7 สิงหานี้ เอากระสุนมึงกลับไป!\nพร...,https://www.facebook.com/FreeYOUTHth/posts/560...,789,6.3 พัน,"[#แค้นมีไว้ชำระ, #ม็อบ7สิงหา, #เอาห่ากระสุนมึง..."
3,2021-07-30 19:32:00,ขณะที่ประเทศกำลังฉิบหาย ผู้คนกำลังล้มตาย โจรกำ...,https://www.facebook.com/FreeYOUTHth/posts/559...,386,766,"[#ม็อบ7สิงหา, #เยาวชนปลดแอก]"
4,2021-07-29 23:07:00,พ.ต.อ.กฤษณะ พัฒนเจริญ รองโฆษกสำนักงานตำรวจแห่ง...,https://www.facebook.com/FreeYOUTHth/posts/559...,3.5 พัน,4 พัน,[#เยาวชนปลดแอก]
5,2021-07-28 20:00:00,ต้องสังเวยด้วยชีวิตของประชาชนอีกเท่าไร จึงจะสำ...,https://www.facebook.com/FreeYOUTHth/posts/558...,1.5 พัน,4.2 พัน,"[#ม็อบ7สิงหา, #เยาวชนปลดแอก]"
6,2021-07-26 22:34:00,ไม่มีแล้วเมืองไทยที่มีสุข\nมีแต่ทุกข์ระทมใจไกล...,https://www.facebook.com/FreeYOUTHth/posts/557...,2.4 พัน,1.3 หมื่น,"[#เยาวชนปลดแอก, #28กรกฎาร่วมใจใส่ชุดดำ]"
7,2021-07-22 21:18:00,ว่อน! เอกสารคล้ายเอกสารราชการจากกรมสารบรรณทหาร...,https://www.facebook.com/FreeYOUTHth/posts/554...,3 พัน,1.4 หมื่น,[#เยาวชนปลดแอก]
8,2021-07-22 15:17:00,หัวใจของประชาธิปไตยคือความเท่าเทียม\n\n\nความเ...,https://www.facebook.com/FreeYOUTHth/posts/554...,76,264,"[#ม็อบ18กรกฎา, #แบนfoodpanda, #สหภาพไรเดอร์, #..."
9,2021-07-21 22:44:00,ทีมเหล่าทัพราษฎรประกาศสนับสนุนข้อเรียกร้องทั้ง...,https://www.facebook.com/FreeYOUTHth/posts/553...,66,348,[#ม็อบ18กรกฎา]


In [5]:
df.to_csv('freeyouth.csv')